In [12]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.stats import sem, t

# Diretório base
directory_base = "/home/nunes/Seismic_Analysis_Tensorflow/results/"
prefix = "LRP_Parihaka_"

def calculate_confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    std_error = sem(data)
    h = std_error * t.ppf((1 + confidence) / 2, n - 1)
    return mean, h

# Resultados consolidados
results = {}

# Percorre as pastas de 1 a 5
for i in range(1, 6):
    folder = os.path.join(directory_base, f"{prefix}{i}", "tables")
    txt_files = glob.glob(os.path.join(folder, "*.txt"))

    for txt_file in txt_files:
        model_name = os.path.basename(txt_file).replace("table_", "").replace(".txt", "")

        # Leitura das métricas no arquivo
        with open(txt_file, "r") as f:
            lines = f.readlines()
            macro_f1 = None
            micro_f1 = None
            
            for idx, line in enumerate(lines):
                if line.startswith("Macro F1"):
                    macro_f1 = float(line.split()[2])
                    micro_f1 = float(lines[idx + 1].split()[2])
                    break

            if macro_f1 is None or micro_f1 is None:
                continue

        # Organiza os resultados por modelo
        if model_name not in results:
            results[model_name] = {"Macro F1": [], "Micro F1": []}

        results[model_name]["Macro F1"].append(macro_f1)
        results[model_name]["Micro F1"].append(micro_f1)

# Calcula média e intervalo de confiança
final_results = []
for model_name, metrics in results.items():
    macro_mean, macro_ci = calculate_confidence_interval(metrics["Macro F1"])
    micro_mean, micro_ci = calculate_confidence_interval(metrics["Micro F1"])
    final_results.append({
        "Model": model_name,
        "Acc Mean": micro_mean,
        "Acc CI": micro_ci,
        "F1 Mean": macro_mean,
        "F1 CI": macro_ci
    })

# Cria tabela final
final_df = pd.DataFrame(final_results)

# Salva a tabela como CSV
output_file = "consolidated_metrics.csv"
final_df.to_csv(output_file, index=False)

print(f"Tabela consolidada salva em {output_file}")


Tabela consolidada salva em consolidated_metrics.csv


In [13]:
import pandas as pd
metrics=pd.read_csv("consolidated_metrics.csv")
metrics

,Model,Acc Mean,Acc CI,F1 Mean,F1 CI
0,UNet3+,0.874308,0.010674,0.738961,0.018282
1,BridgeNet,0.869432,0.011396,0.720817,0.024250
2,ENet,0.860699,0.010273,0.729141,0.014586
3,ESPNet,0.681058,0.157499,0.416129,0.115517
4,UNet,0.872417,0.006135,0.733698,0.004558
5,EfficientNetB1,0.856784,0.007591,0.714847,0.009437
6,Attention,0.865695,0.015722,0.723045,0.032171
7,LWBNA,0.337800,0.127121,0.132867,0.072654
8,CPFNetM,0.848055,0.020198,0.706840,0.019577
9,ICNet,0.863510,0.008714,0.735303,0.007884


In [1]:
import platform
import psutil
import os
import subprocess

# Collecting machine info
system_info = {
    "OS": platform.system(),
    "OS Version": platform.version(),
    "OS Release": platform.release(),
    "CPU": platform.processor(),
    "Cores": psutil.cpu_count(logical=False),
    "Logical CPUs": psutil.cpu_count(logical=True),
    "RAM (GB)": round(psutil.virtual_memory().total / (1024 ** 3), 2),
}

# Collecting GPU info (if NVIDIA GPU is installed)
gpu_info = "Not Available"
try:
    gpu_info = subprocess.check_output(
        "nvidia-smi --query-gpu=name --format=csv,noheader", shell=True, text=True
    ).strip()
except Exception:
    pass

system_info["GPU"] = gpu_info

system_info


{'OS': 'Linux',
 'OS Version': '#48-Ubuntu SMP PREEMPT_DYNAMIC Fri Sep 27 14:04:52 UTC 2024',
 'OS Release': '6.8.0-48-generic',
 'CPU': 'x86_64',
 'Cores': 12,
 'Logical CPUs': 24,
 'RAM (GB)': 61.98,
 'GPU': 'NVIDIA GeForce RTX 4090\nNVIDIA GeForce RTX 4090'}